In [41]:
!pip install numba
!pip install --upgrade tbb

In [42]:
from numba import njit, jit, vectorize, float64, cuda, prange
import numba
import numpy as np

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['vectorize', 'float64']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [43]:
numba.cuda.detect()

Found 1 CUDA devices
id 0            b'Tesla K80'                              [SUPPORTED]
                      compute capability: 3.7
                           pci device id: 4
                              pci bus id: 0
Summary:
	1/1 devices are supported


True

In [44]:
x = 256

In [49]:
a = np.random.randn(x, x)
b = np.random.randn(x, x)
c = np.zeros((x, x))

In [50]:
def cpuMM(a, b, c):
      for i in range(c.shape[0]):
        for j in range(c.shape[1]):               
            tmp = 0
            for k in range(a.shape[1]):
                tmp += a[i, k] * a[k, j] 
            c[i, j] = tmp 
host_mat1 = c
%time _= cpuMM(a, b, c)


CPU times: user 10.5 s, sys: 31.9 ms, total: 10.5 s
Wall time: 10.5 s


In [52]:
@cuda.jit
def gpuMM(a, b, c):
  i, j = cuda.grid(2)
  tmp = 0
  for k in range(a.shape[1]):
    tmp += a[i, k] * b[k, j]
  c[i, j] = tmp

count_threads = 6

def DeviceHost(a, b, c):
  device_A = cuda.to_device(a)
  device_B = cuda.to_device(b)
  device_C = cuda.device_array(c.shape, np.float64)

  threads_per_block = (count_threads, count_threads)
  blocks_per_grid_x = math.ceil(a.shape[0]/threads_per_block[0])
  blocks_per_grid_y = math.ceil(a.shape[1]/threads_per_block[1])
  blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

  gpuMM[blocks_per_grid, threads_per_block](device_A, device_B, device_C)
  return device_C.copy_to_host()
%time _= DeviceHost(a, b, c)
dev_mat = c


CPU times: user 194 ms, sys: 3 ms, total: 197 ms
Wall time: 196 ms


In [54]:
print(np.allclose(dev_mat, host_mat1))

True
